In [1]:
import pandas as pd
import json

In [2]:
rappi = pd.read_csv('../scraper/data/rappi_revolucion258_morning.csv')
eats = pd.read_csv('../scraper/data/ue_revolucion258_morning.csv')

In [3]:
rappi.head(3)

,id,name,lat,lng,cuisine,platform
0,ojo-de-agua,Ojo de Agua Condesa Express,19.409313,-99.171596,"Saludable, Sándwiches",rappi
1,le-pain-quotidien,Le Pain Quotidien Nueva York,19.389100,-99.175991,"Desayuno, Saludable, Sándwiches",rappi
2,peltre-loncheria,Peltre Lonchería - Condesa,19.407664,-99.174414,"Desayuno, Mexicana, Saludable",rappi


In [4]:
eats.head(3)

,id,name,lat,lng,cuisine,platform
0,starbucks-wtc,Starbucks (WTC),19.392584,-99.173313,Café y té,eats
1,el-bajio-comida-mexicana-insurgentes,El Bajío Comida Mexicana (Insurgentes),19.390169,-99.173355,Mexicana,eats
2,churreria-porfirio-poliforum,Churrería Porfirio (Poliforum),19.391891,-99.172061,Postres,eats


In [5]:
RES = 11

In [6]:
def df_map(df_restos, res):
    df = df_restos[['id', 'platform', 'lat', 'lng']].reset_index(drop=True).copy()
    df['hex_id'] = df.apply(lambda row: h3.geo_to_h3(row['lat'], row['lng'], res=res), axis=1)
    
    polygons = []
    for h in df.hex_id:
        polygons.append({
            "type" : "Polygon"
            , "coordinates": [h3.h3_to_geo_boundary(h3_address=h, geo_json=True)]
        })
    df['geometry'] = pd.Series(polygons)
    df['res'] = res
    return(df)

In [7]:
from h3 import h3
from geojson.feature import *
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster

In [8]:
map_example = Map(
        location= [19.3987204, -99.184695]
        , zoom_start=16
        , tiles="cartodbpositron"
    )

In [9]:
cloud_kitchen_hex = h3.geo_to_h3(19.3987204, -99.184695, res=RES)
ck_feature = Feature(
    geometry = {
        "type" : "Polygon"
        , "coordinates": [h3.h3_to_geo_boundary(h3_address=cloud_kitchen_hex, geo_json=True)]
    }
    , id = cloud_kitchen_hex
    , properties = {"name": "CloudKitchen", "resolution": int(RES)}
)

ck_geojson = json.dumps(FeatureCollection([ck_feature]))

In [10]:
cloud_kitchen_hex = h3.geo_to_h3(19.3987204, -99.184695, res=RES)
ck_feature = Feature(
    geometry = {
        "type" : "Polygon"
        , "coordinates": [h3.h3_to_geo_boundary(h3_address=cloud_kitchen_hex, geo_json=True)]
    }
    , id = cloud_kitchen_hex
    , properties = {"name": "CloudKitchen", "resolution": int(RES)}
)

ck_geojson = json.dumps(FeatureCollection([ck_feature]))

In [11]:
list_features = []
df_eats_map = df_map(eats, RES)
for i, row in df_eats_map.iterrows():
    feature = Feature(
        geometry = row["geometry"]
        , id=row["hex_id"]
        , properties = {"resolution": int(row["res"])}
    )
    list_features.append(feature)

eats_geojson = json.dumps(FeatureCollection(list_features))

list_features = []
df_rappi_map = df_map(rappi, RES)
for i, row in df_rappi_map.iterrows():
    feature = Feature(
        geometry = row["geometry"]
        , id=row["hex_id"]
        , properties = {"resolution": int(row["res"])}
    )
    list_features.append(feature)


rappi_geojson = json.dumps(FeatureCollection(list_features))

In [12]:
GeoJson(
    ck_geojson
    , style_function = lambda feature: {
        'fillColor': "blue"
        , 'weight': 2
        , 'fillOpacity': 1
    }
).add_to(map_example)

GeoJson(
    eats_geojson
    , style_function = lambda feature: {
        'fillColor': "green"
        , 'color': "green"
        , 'weight': 2
        , 'fillOpacity': 0.4
    }
    , name = "Example" 
).add_to(map_example)

GeoJson(
    rappi_geojson
    , style_function = lambda feature: {
        'fillColor': "orange"
        , 'color': "orange"
        , 'weight': 2
        , 'fillOpacity': 0.4
    }
    , name = "Example" 
).add_to(map_example)

map_example

In [24]:
df_hexs = df_eats_map.merge(df_rappi_map, on='hex_id', how='outer')

In [25]:
df_hexs = (
    df_hexs
    [['hex_id', 'id_x', 'id_y']]
    .rename(columns={
        'id_x': 'eats_id'
        , 'id_y': 'rappi_id'
    })
)

In [26]:
print('Total Unique Hexs:', len(df_hexs.hex_id.unique()))
print('Total Rows:', len(df_hexs))

Total Unique Hexs: 592
Total Rows: 748


In [27]:
df_hexs[['hex_id', 'eats_id', 'rappi_id']].sample(10)

,hex_id,eats_id,rappi_id
565,8b4995bae5b0fff,NaN,pancracia
664,8b4995ba3b2efff,NaN,ala-chicken
639,8b499584bca0fff,NaN,pasteleria-galette
452,8b4995ba3addfff,cito,NaN
712,8b4995ba568cfff,NaN,little-caesars
205,8b4995ba02f0fff,kitchen-lab,boicot-cafe
325,8b4995ba3315fff,cafe-49,NaN
530,8b4995ba30a8fff,NaN,tacos-y-deli
372,8b499584a6c9fff,lunch2go,NaN
560,8b4995ba0ca1fff,NaN,amores-tres23


In [28]:
df_agg_hex = (
    df_hexs
    [['hex_id', 'eats_id', 'rappi_id']]
    .groupby('hex_id')
    .count()
    .reset_index()
)

In [30]:
def which_platform(eats, rappi):
    if (0 < eats) and (0 < rappi):
        return 'both'
    elif (eats == 0) and (0 < rappi):
        return 'rappi'
    elif (0 < eats) and (rappi == 0):
        return 'eats'

In [31]:
df_agg_hex['platform'] = df_agg_hex.apply(lambda row: which_platform(row['eats_id'], row['rappi_id']), axis=1)
df_agg_hex['total_restos'] = df_agg_hex.apply(lambda row: row['eats_id'] + row['rappi_id'], axis=1)

In [36]:
print('Total Unique Hexs:', len(df_agg_hex))
df_agg_hex.sample(5)

Total Unique Hexs: 592


,hex_id,eats_id,rappi_id,platform,total_restos
431,8b4995ba4603fff,28,28,both,56
569,8b4995bae835fff,0,1,rappi,1
375,8b4995ba3c85fff,1,0,eats,1
407,8b4995ba4144fff,2,0,eats,2
434,8b4995ba4630fff,0,1,rappi,1


In [38]:
df_agg_hex.sort_values('total_restos', ascending=False).head(10)

,hex_id,eats_id,rappi_id,platform,total_restos
484,8b4995ba5792fff,35,35,both,70
431,8b4995ba4603fff,28,28,both,56
347,8b4995ba3901fff,5,5,both,10
4,8b499584b119fff,0,9,rappi,9
118,8b4995ba0744fff,8,0,eats,8
365,8b4995ba3b2efff,0,8,rappi,8
278,8b4995ba309afff,4,4,both,8
564,8b4995bae5b0fff,0,5,rappi,5
148,8b4995ba0906fff,3,2,both,5
255,8b4995ba2872fff,2,2,both,4


In [50]:
df_not_dup = df_agg_hex[df_agg_hex.platform != 'both'][['hex_id', 'platform']].merge(df_hexs)
print('Total Unique Hexs:', len(df_not_dup.hex_id.unique()))
print('Total Rows:', len(df_not_dup))

Total Unique Hexs: 534
Total Rows: 611


In [58]:
hex_not_dup = df_not_dup.hex_id.unique().tolist()

In [61]:
df_eats_unique = df_eats_map[df_eats_map.hex_id.isin(hex_not_dup)]
df_rappi_unique = df_rappi_map[df_rappi_map.hex_id.isin(hex_not_dup)]

In [65]:
df_restos = pd.concat([df_eats_unique, df_rappi_unique], axis=0)

In [80]:
df_restos.sample(1).geometry.iloc[0]

{'type': 'Polygon',
 'coordinates': [[[-99.16645299649885, 19.390988053930062],
   [-99.16627581899711, 19.39121266452004],
   [-99.16639015220522, 19.391467981550065],
   [-99.16668166393049, 19.391498687768216],
   [-99.16685884146142, 19.391274076477583],
   [-99.16674450723792, 19.391018759669457],
   [-99.16645299649885, 19.390988053930062]]]}

In [123]:
m = folium.Map(
    [19.3987204, -99.184695]
    , zoom_start=16
    , tiles="cartodbpositron"
)

GeoJson(
    ck_geojson
    , style_function = lambda feature: {
        'fillColor': "blue"
        , 'weight': 2
        , 'fillOpacity': 1
    }
).add_to(m)

fg = folium.map.FeatureGroup().add_to(m)

list_features = []
for i, row in df_restos.iterrows():
    feature = Feature(
        geometry = row["geometry"]
        , id=row["hex_id"]
        , properties = {
            "resolution": int(row["res"])
            , "platform": row["platform"]
            , "@id": row["id"]
        }
    )
    list_features.append(feature)

restos_geojson = json.dumps(FeatureCollection(list_features))

for f in json.loads(restos_geojson)['features']:
    geo = GeoJson(
        f
        , style_function = lambda feature: {
            'fillColor': None
            , 'color': "green" if feature["properties"]["platform"] == "eats" else "orange"
            , 'weight': 2
            , 'fillOpacity': 0.4
        }
        , name = "Example" 
    )
    geo.add_child(folium.Popup(f['properties']['@id']))
    geo.add_to(m)

m

In [51]:
df_dup = df_agg_hex[df_agg_hex.platform == 'both'][['hex_id', 'platform']].merge(df_hexs)
print('Total Unique Hexs:', len(df_dup.hex_id.unique()))
print('Total Rows:', len(df_dup))

Total Unique Hexs: 58
Total Rows: 137


In [53]:
df_dup.sample(5)

,hex_id,platform,eats_id,rappi_id
60,8b4995ba4603fff,both,el-huarache-del-valle,bacon-eggs
76,8b4995ba4603fff,both,desayunos-y-comidas,bacon-eggs
110,8b4995ba5792fff,both,las-ahogadas-de-tlaquepaque,botanero-campestre
44,8b4995ba3901fff,both,senor-taco-condesa,senor-taco
56,8b4995ba4603fff,both,tacos-de-canasta-eventos,bacon-eggs


In [55]:
df_dup.groupby('hex_id').count().sort_values('platform', ascending=False).head(3)

,platform,eats_id,rappi_id
hex_id,,,
8b4995ba5792fff,35,35,35
8b4995ba4603fff,28,28,28
8b4995ba3901fff,5,5,5


In [56]:
df_dup[df_dup.hex_id == '8b4995ba5792fff']

,hex_id,platform,eats_id,rappi_id
91,8b4995ba5792fff,both,donas-americanas-desde-1980-condesa,roco-gelato
92,8b4995ba5792fff,both,donas-americanas-desde-1980-condesa,las-ahogadas-de-tlaquepaque
93,8b4995ba5792fff,both,donas-americanas-desde-1980-condesa,bucaneras
94,8b4995ba5792fff,both,donas-americanas-desde-1980-condesa,taco-burro-fast-food
95,8b4995ba5792fff,both,donas-americanas-desde-1980-condesa,taqueria-campestre
96,8b4995ba5792fff,both,donas-americanas-desde-1980-condesa,botanero-campestre
97,8b4995ba5792fff,both,donas-americanas-desde-1980-condesa,flautas
98,8b4995ba5792fff,both,huarache-rafa-los-tradicionales-desde-1979,roco-gelato
99,8b4995ba5792fff,both,huarache-rafa-los-tradicionales-desde-1979,las-ahogadas-de-tlaquepaque
100,8b4995ba5792fff,both,huarache-rafa-los-tradicionales-desde-1979,bucaneras


In [ ]:
rappi_contains_eats = df_shared.apply(lambda row: True if row['eats_id'] in row['rappi_id'] else False, axis = 1)
eats_contains_rappi = df_shared.apply(lambda row: True if row['rappi_id'] in row['eats_id'] else False, axis = 1)

In [ ]:
df_shared[~(eats_contains_rappi | rappi_contains_eats)]

In [ ]:
df_shared[df_shared.hex_id == '8b4995ba5792fff']